# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [205]:
def build_kernel(data_set, kernel_type='linear', d=3):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = np.power(kernel, d)
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [216]:
from sklearn.model_selection import train_test_split
import numpy as np
import cvxopt

def choose_set_for_label(data_set, label):
    data = data_set.data
    labels = data_set.target

    binary_labels = np.where(labels == label, 1, -1)

    train_data_set, test_data_set, train_labels, test_labels = train_test_split(
        data, binary_labels, test_size=0.2, random_state=15)

    return train_data_set, test_data_set, train_labels, test_labels

In [217]:
def get_labels_count(data_set):
    labels_count = len(np.unique(data_set))
    return labels_count

Use the code that we have implemented earlier:

In [222]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    objects_count = len(train_data_set)
    
    P = cvxopt.matrix(np.outer(train_labels, train_labels) * kernel)
    q = cvxopt.matrix(-np.ones((objects_count, 1)))
    G = cvxopt.matrix(np.vstack((np.eye(objects_count) * -1, np.eye(objects_count))))
    h = cvxopt.matrix(np.hstack((np.zeros(objects_count), np.ones(objects_count) * C)))
    A = cvxopt.matrix(train_labels, (1, objects_count), 'd')
    b = cvxopt.matrix(0.0)

    cvxopt.solvers.options['show_progress'] = False
    
    sol = cvxopt.solvers.qp(P, q, G, h, A, b)
    
    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]
    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]
    
    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    if len(lambdas) > 0:
        b /= len(lambdas)
    else:
        b = 0

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = np.dot(test_data_set, support_vectors.T)
    sigma = 1.0
    #K = np.dot(test_data_set, support_vectors.T)
    #kernel = build_kernel(train_data_set, kernel_type='rbf')
    c = (1. / sigma * np.sum(test_data_set ** 2, axis=1) * np.ones((1, np.shape(test_data_set)[0]))).T
    c = np.dot(c, np.ones((1, np.shape(kernel)[1])))
    #aa = np.dot((np.diag(K)*np.ones((1,len(test_data_set)))).T[support_vectors_id], np.ones((1, np.shape(K)[0]))).T
    sv = (np.diag(np.dot(train_data_set, train_data_set.T))*np.ones((1,len(train_data_set)))).T[support_vectors_id]
    aa = np.dot(sv,np.ones((1,np.shape(kernel)[0]))).T
    kernel = kernel - 0.5 * c - 0.5 * aa
    kernel = np.exp(kernel / (2. * sigma ** 2))

    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

In [226]:
# modify this part 
from sklearn.datasets import load_iris
iris = load_iris()
label = 2

train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(iris, label)
objects_count = len(train_labels)

lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='poly')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

0.7
